In [ ]:
import torch
import matplotlib.pyplot as plt
from transformers import StoppingCriteria, StoppingCriteriaList

from m2_utilities.load_data import load_trajectories
from m2_utilities.preprocessor import Preprocessor
from m2_utilities.data.postprocessor import batch_truncate_sequence
from m2_utilities.qwen import load_qwen
from m2_utilities.metrics import forecast_points, compute_mae

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [37]:
model, tokenizer = load_qwen()

In [ ]:
trajectories = load_trajectories("data/lotka_volterra_data.h5")[:, :, :]
input_points = trajectories[:, :20, :]
preprocessor = Preprocessor(decimals=2)
input_ids = preprocessor.encode(trajectories)

print(input_ids.shape)

torch.Size([2, 999])


In [49]:
forecast = forecast_points(model, trajectories, 1, 2)

In [50]:
print(forecast)

tensor([[[0.6800, 0.5600]],

        [[1.2600, 0.6900]]])
